# Lab 8. Iteratory, generatory oraz wyrażenia generujące

### 1. Iteratory i generatory

### 1.1. Iteratory

> * Dokumentacja PEP: https://peps.python.org/pep-0234/
> * Pojęcia iteratora w dokumentacji Python: https://docs.python.org/3/glossary.html#term-iterator

Rozpatrując poniższy fragment kodu:
```python
for element in range(1, 11):
    print(element)
```

wszystko raczej jest jasne. Ale skąd pętla for wie jak ma się uniwersalnie zachowywać dla różnych obiektów iterowalnego ? Cały mechanizm jest obsługiwany przez iteratory. W niewidoczny dla nas sposób pętle for wywołuje funkcję `iter()` na obiekcie kolekcji. Funkcja zwraca obiekt iteratora, który ma zdefiniowaną metodę `__next__()`, odpowiedzialną za zwracanie kolejnych elementów kolekcji. Kiedy nie ma już więcej elementów kolekcji zgłaszany jest wyjątek `StopIteration`, kończący działanie pętli for. Można wywołać funkcję `__next__()` iteratora za pomocą wbudowanej funkcji `next()`.

**Przykład:**
```python
imie = "Reks"
it = iter(imie)
print(it)
# na wyjściu: <str_iterator object at 0x0000000003807FD0>
next(it)
# 'R'
next(it)
# 'e'
next(it)
# 'k'
next(it)
# 's'
next(it)
# Traceback (most recent call last):
#  File "<input>", line 1, in <module>
# StopIteration
```

Przykład implementacji własnego iteratora.

In [19]:
class Wspak:
    """Iterator zwracający wartości w odwróconym porządku"""
    
    def __init__(self, data):
        self.data = data
        self.index = len(data)

    def __iter__(self):
        return self

    def __next__(self):
        if self.index == 0:
            raise StopIteration
        self.index = self.index - 1
        return self.data[self.index]

W laboratorium numer 3 przedstawiona została koncepcja abstrakcyjnych klas bazowych oraz zbiór wbudowanych klas abstrakcyjnych dla modułu collections. Wśród innych klas znajdowały się dwie, które nawiązują bezpośrednio do iteratorów: `collections.abc.Iterator` oraz `collections.abc.Iterable`. 

> Dokumentacja dla przypomnienia: https://docs.python.org/3/library/collections.abc.html

Klasa `Iterable` zapewnia implementację metody `__iter__`, a klasa `Iterator` zarówno metodę `__iter__` oraz `__next__`.

Przetestowanie czy obiekt jest iterowalny czy nie można wykonać za pomocą wbudowanych metod `isinstance()` oraz `issubclass()` albo poprzez opakowanie rzutowania obiektu na iterator poprzez wywołanie `iter(obiekt)` i obsłużenie wyjątku `TypeError`.

In [9]:
# próba rzutowania
try:
    iter(4)
except TypeError as e:
    print(e)

'int' object is not iterable


In [5]:
import collections.abc

some_types_to_check = [str, int, tuple, list, dict, set]

def check(objects_to_check):
    for obj in objects_to_check:
        print(f'Obiekt {obj} dziedziczy po collections.abc.Iterable: {issubclass(obj, collections.abc.Iterable)}')
        print(f'Obiekt {obj} dziedziczy po collections.abc.Iterator: {issubclass(obj, collections.abc.Iterator)}')
        print(f'Obiekt {obj} posiada metodę __next__: {hasattr(obj, '__next__')}')
        print(f'Obiekt {obj} posiada metodę __iter__: {hasattr(obj, '__iter__')}')
        # rzutowanie na obiekt iteratora
        try:
            # wywołujemy (nag. call) obiekty zapisane na liście
            obj_iter = iter(obj())
            print(f'Obiekt iteratora dla obiektu {obj} to {obj_iter.__class__.__name__}')
        except TypeError as e:
            print("Rzutowanie zakończone błędem!")
            print(e)
        print('-' * 20)

In [6]:
# testujemy obiekty wg. zaimplementowanej logiki
check(some_types_to_check)

Obiekt <class 'str'> dziedziczy po collections.abc.Iterable: True
Obiekt <class 'str'> dziedziczy po collections.abc.Iterator: False
Obiekt <class 'str'> posiada metodę __next__: False
Obiekt <class 'str'> posiada metodę __iter__: True
Obiekt iteratora dla obiektu <class 'str'> to str_ascii_iterator
--------------------
Obiekt <class 'int'> dziedziczy po collections.abc.Iterable: False
Obiekt <class 'int'> dziedziczy po collections.abc.Iterator: False
Obiekt <class 'int'> posiada metodę __next__: False
Obiekt <class 'int'> posiada metodę __iter__: False
Rzutowanie zakończone błędem!
'int' object is not iterable
--------------------
Obiekt <class 'tuple'> dziedziczy po collections.abc.Iterable: True
Obiekt <class 'tuple'> dziedziczy po collections.abc.Iterator: False
Obiekt <class 'tuple'> posiada metodę __next__: False
Obiekt <class 'tuple'> posiada metodę __iter__: True
Obiekt iteratora dla obiektu <class 'tuple'> to tuple_iterator
--------------------
Obiekt <class 'list'> dziedziczy

Jak widać na wyjściu powyższej komórki, każdy z obiektów, który jest iterowalny, jest rzutowalny na obiekt iteratora, ale nie jest to klasa ogólnego przeznaczenia. W Pythonie mamy dedykowane implementacje iteratorów dla różnych typów wbudowanych.

Widać również, że klasy takie jak `str, tuple, set, list` nie są iteratorami, ale mówimy, że są iterowalnymi kontenerami.

Pamiętaj, że iteratory są wyczerpywalne, co oznacza, że po ich wykorzystaniu nie można ich ponownie użyć bez utworzenia nowego obiektu.

Wiemy już, że metody `__iter__` oraz `__next__` są elementem API Pythona, które pozwalają na implementację funkcjonalności związanej z iteratorami, ale to jeszcze nie wszystko co otrzymujemy w standardowej bibliotece. Wszystkie klasy, które dostarczają funkcji magicznej `__getitem__` również można rzutować na iterator i wykorzystać w ten sam sposób jak pokazano w poprzednich przykładach. Poniżej przykład klasy, która nie posiada metod  `__iter__` oraz `__next__`, ale posiada za to metodę `__getitem__`.

In [14]:
class Vector2d:

    def __init__(self):
        self.values = [0,0]

    def set_values(x,y):
        self.values[0] = x
        self.values[1] = y

    def __getitem__(self, idx):
        if idx >= len(self.values):
            raise IndexError
        return self.values[idx]

In [15]:
check([Vector2d])

Obiekt <class '__main__.Vector2d'> dziedziczy po collections.abc.Iterable: False
Obiekt <class '__main__.Vector2d'> dziedziczy po collections.abc.Iterator: False
Obiekt <class '__main__.Vector2d'> posiada metodę __next__: False
Obiekt <class '__main__.Vector2d'> posiada metodę __iter__: False
Obiekt iteratora dla obiektu <class '__main__.Vector2d'> to iterator
--------------------


In [18]:
vec = Vector2d()
vector_iterator = iter(vec)
print(next(vector_iterator))
print(next(vector_iterator))

0
0


In [43]:
# przykład implementacji iteratora dla skończonego ciągu liczb Fibonacciego
class FibonacciIterator:
    def __init__(self, limit):
        self.limit = limit
        self.a, self.b = 0, 1

    def __iter__(self):
        return self

    def __next__(self):
        if self.a > self.limit:
            raise StopIteration

        fibonacci_number = self.a
        self.a, self.b = self.b, self.a + self.b

        return fibonacci_number

In [46]:
fib_iter = FibonacciIterator(100)
for fib in fib_iter:
    print(fib)

0
1
1
2
3
5
8
13
21
34
55
89


### 1.2 Funkcja generatora

**Dokumentacja:**
* https://wiki.python.org/moin/Generators
* https://peps.python.org/pep-0255/


Generatory są prostymi narzędziami do tworzenia iteratorów. Generatory piszemy jak standardowe funkcje, ale zamiast instrukcji `return` używamy `yield`, która wydaje się tutaj bardzo podobna w działaniu, ale są znaczne różnice. Za każdym razem kiedy funkcja `next()` jest wywoływana na generatorze wznawia on swoje działanie w momencie, w którym został przerwany instrukcją `yield`. Instrukcja `yield` zwraca obiekt generatora, mówi się również `daje` lub `produkuje` wartości. Przy próbie wywołania `next` na obiekcie generatora, który wyczerpał już produkcję wartości zgłaszany jest wyjątek `StopIteration`.

Każda funkcja w języku Python, która posiada instrukcję `yield` jest nazywana `funkcją generatora`.

Od wersji 3.5 Pythona mamy również do dyspozycji generatory asynchroniczne (poprzez `async def`), ale ten temat został tutaj pominięty.

Poniżej przykład generatora, którego działanie jest podobne do iteratora zaprezentowanego w przykładzie.

In [20]:
def reverse(data):
    for index in range(len(data)-1, -1, -1):
        yield data[index]


gen = reverse("Feliks")
print(next(gen))
print("Marek")
print(next(gen))

s
Marek
k


Możliwe jest wykorzystanie wielu instrukcji `yield` w ciele jednej funkcji generującej.

In [22]:
def gen_tic_tac_toe():
    yield 'TIC'
    yield 'TAC'
    yield 'TOE'

In [24]:
gen = gen_tic_tac_toe()
print(next(gen))
print('Kolejny krok...')
print(next(gen))
print('Kolejny krok...')
print(next(gen))

print(next(gen))

TIC
Kolejny krok...
TAC
Kolejny krok...
TOE


StopIteration: 

### 1.3 Wyrażenie generujące

Podobnie do wyrażeń listowych (Python comprehension) możliwe jest również zapisanie wyrażenia generatora w analogiczny sposób. Używamy do tego celu **nawiasów zwykłych**. Przykład pniżej.

In [27]:
# wyrażenia generujące
litery = (litera for litera in "Zdzisław")
print(litery)
print(next(litery))
print(next(litery))
print(next(litery))

# i cała reszta
print(list(litery))

<generator object <genexpr> at 0x000001480791C100>
Z
d
z
['i', 's', 'ł', 'a', 'w']


## Zadania

**Zadanie 1**

Napisz własny iterator, który będzie zwracał tylko elementy z parzystych indeksów przekazanej sekwencji.

**Zadanie 2**

Bazując na przykładzie z iteratorem generującym kolejne wartości ciągu Fibonacciego napisz iterator, który generuje liczby pierwsze.

**Zadanie 3**

Napisz iterator, który zwraca nazwy dni tygodnia w języku polskim (patrz zadanie 4, lab 7). Iterator inicjalizujemy indeksem wskazującym, od którego dnia iteracja się rozpoczyna. Iterator powinien działać w sposób nieskończony (ale uważaj w trakcie jego testowania).

**Zadanie 4**

Napisz iterator, który będzie zwracał kolejne słowa z przekazanego tekstu, ale wykorzystaj wyrażenia regularne do wydobycia tych słów. Postaraj się wykorzystać iterator również dla znalezionych dopasowań dla tego wyrażenia (patrz poprzednie laboratoria).


**Zadanie 5**

Przepisz iterator z zadania 4 na generator (funkcja generująca).


**Zadanie 6**

Napisz generator kodów produktów, który przyjmuje dwa argumenty inicjujące: `letter_pos`, `num_pos` - oba są typem `int`. Ten generator ma zwracać kolejny kod produktu według schematu: 

* wywołanie dla `letter_pos = 1` oraz `num_pos = 2` generuje kody od `A_01` do `Z_99`
* wywołanie dla `letter_pos = 2` oraz `num_pos = 3` generuje kody od `A_001` do `ZZ_999`

Rzuć okiem na moduł `string` oraz mmoduł `itertools` z poprzedniego laboratorium, aby wykorzystać funkcje pomocnicze.